In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.18.0'

In [14]:
class FileDataset(tf.data.Dataset):
  def read_files_in_batches(num_samples):
    # Opening the file
    time.sleep(0.03)
    for sample_idx in range(num_samples):
      time.sleep(0.015)
      yield (sample_idx,)

  def __new__(cls,num_samples=3):
    return tf.data.Dataset.from_generator(
        cls.read_files_in_batches,
        output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
        args=(num_samples,)
    )
    pass

In [12]:
def benchmark(dataset, num_epochs=2):
  for epoch_num in range(num_epochs):
    for sample in dataset:
      time.sleep(0.01)

In [21]:
%%timeit
benchmark(FileDataset())

255 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
benchmark(FileDataset().prefetch(1))

268 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

260 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
dataset=tf.data.Dataset.range(5)
for d in dataset:
  print(d.numpy())

0
1
2
3
4


In [30]:
dataset=dataset.map(lambda x:x**2)

In [31]:
def mapped_function(s):
  tf.py_function(lambda :time.sleep(0.03),[],())
  return s

In [33]:
%%timeit
benchmark(FileDataset().map(mapped_function),5)

1.1 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%%timeit
benchmark(FileDataset().map(mapped_function).cache(),5)

395 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
